Most of the code is from [Tutorial from Depends on The Definition](https://www.depends-on-the-definition.com/named-entity-recognition-with-bert/) but modified for this final project task and domain.

# Setup and Import Library

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
%notebook inline

In [3]:
ROOT_PATH = '/content/drive/My Drive/Tugas/Tugas Semester 8/Tugas Akhir/13516152 - Deborah Aprilia Josephine/';

In [4]:
!pip install transformers

     |████████████████████████████████| 778kB 2.8MB/s 
     |████████████████████████████████| 890kB 9.4MB/s 
     |████████████████████████████████| 1.1MB 16.7MB/s 
     |████████████████████████████████| 3.0MB 17.5MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=702c99ddfb5cfc988dabb73d32346b7059a16d9ff3c26a16da2e2fba1fd36de5
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [5]:
!pip install seqeval

  Created wheel for seqeval: filename=seqeval-0.0.12-cp36-none-any.whl size=7424 sha256=125741013791b6c144128385602b7011ca1f33de937baff465146bc771c3bb91
  Stored in directory: /root/.cache/pip/wheels/4f/32/0a/df3b340a82583566975377d65e724895b3fad101a3fb729f68
Successfully built seqeval


In [6]:
import pandas as pd 
import numpy as np
import json

import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertTokenizer, BertConfig, XLMRobertaTokenizer, XLMRobertaConfig, DistilBertTokenizer, DistilBertConfig

from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

import transformers
from transformers import AdamW

from seqeval.metrics import f1_score, accuracy_score
from seqeval.metrics import classification_report

transformers.__version__
torch.__version__

'1.6.0+cu101'

In [7]:
model_type = "mbert"
FINE_TUNING_TYPE = "full"
VERSION = "final"
SCENARIO = 3
EPOCH_CONST = 25
BATCH_SIZE = 8
WEIGHT_DECAY_RATE = 0.01
TRAINING_SIZE = None
TESTING_SIZE = None

In [8]:
experiment_information = model_type + "_v" + str(VERSION)
if FINE_TUNING_TYPE == "full" :
  experiment_information += "_full"
elif FINE_TUNING_TYPE == "partial" :
  experiment_information += "_partial"

print ("Info", experiment_information)

MODEL_PATH = ROOT_PATH + "model/scenario " + str(SCENARIO) + '/'
RESULT_PATH = ROOT_PATH + "result/scenario " + str(SCENARIO) + '/'
DATASET_PATH = ROOT_PATH + "dataset/version " + str(VERSION) + '/'
ANALYZE_PATH = ROOT_PATH + "analyze/version " + str(VERSION) + '/'
MODEL_FILENAME = experiment_information + ".pth"

print (MODEL_PATH)
print (RESULT_PATH)
print (DATASET_PATH)

Info mbert_vfinal_full
/content/drive/My Drive/Tugas/Tugas Semester 8/Tugas Akhir/13516152 - Deborah Aprilia Josephine/model/scenario 3/
/content/drive/My Drive/Tugas/Tugas Semester 8/Tugas Akhir/13516152 - Deborah Aprilia Josephine/result/scenario 3/
/content/drive/My Drive/Tugas/Tugas Semester 8/Tugas Akhir/13516152 - Deborah Aprilia Josephine/dataset/version final/


# Dataset

In [9]:
def read_json(filename) :
  with open(filename, 'r', encoding="utf8") as f:
    obj = json.load(f)
  return obj

def write_json(obj, filename) :
  with open(filename, 'w', encoding="utf8") as outfile:
    json.dump(obj, outfile)
  print ("Successfully write JSON obj to", filename)

def write_tsv(obj, filename) :
  file = open(filename, "w", encoding="utf-8") 
  for tokens in obj :
    for token in tokens :
      file.write(token['token'] + "\t" + token['label'] + "\n")
    file.write("==\n\n")
  file.close()
  print ("Successfully write JSON obj to", filename)

def read_tsv(filename) :
  file = open(filename, "r") 
  obj = {}
  items = []
  words = file.read().split("==\n\n")
  sentences = [[tokens.split("\t") for tokens in word.split("\n")] for word in words]
  for tokens in sentences :
    item = []
    for token in tokens :
      if len(token) == 2 :
        tok_lab = {}
        tok_lab['token'] = token[0]
        tok_lab['label'] = token[1]
        item.append(tok_lab)
    if len(item) > 1 :
      items.append(item)
  obj['tokens_labels'] = items
  return obj

def write_to_txt(obj, filename, key=None) :
  file = open(filename, "w", encoding="utf-8") 
  for items in obj :
      texts = []
      for item in items :
        if key :
          texts.append(item[key])
        else :
          texts.append(item)
      file.write(" ".join(texts) + "\n")
  file.close()
  print ("Successfully write JSON obj to", filename)

def read_txt(filename) :
  results = open(filename, "r").readlines()
  return results

In [10]:
LIMIT_SIZE = 5

In [11]:
tag2idx = {
    "1-B-Merek" : 1,
    "1-I-Merek" : 2,
    "2-O" : 3,
    "3-B-NamaProduk" : 4,
    "3-I-NamaProduk" : 5,
    "4-B-Varian" : 6,
    "4-I-Varian" : 7,
    "5-B-Ukuran" : 8,
    "5-I-Ukuran" : 9,
    "6-B-Penggunaan" : 10,
    "6-I-Penggunaan" : 11,
    "7-B-Tekstur" : 12,
    "7-I-Tekstur" : 13,
    "PAD" : 14
}

tag2idx2= {
    "B-Merek" : 1,
    "I-Merek" : 2,
    "O" : 3,
    "B-NamaProduk" : 4,
    "I-NamaProduk" : 5,
    "B-Varian" : 6,
    "I-Varian" : 7,
    "B-Ukuran" : 8,
    "I-Ukuran" : 9,
    "B-Penggunaan" : 10,
    "I-Penggunaan" : 11,
    "B-Tekstur" : 12,
    "I-Tekstur" : 13,
    "PAD" : 14
}

tag_values = [tag for tag in tag2idx2.keys()]
print (tag_values)

['B-Merek', 'I-Merek', 'O', 'B-NamaProduk', 'I-NamaProduk', 'B-Varian', 'I-Varian', 'B-Ukuran', 'I-Ukuran', 'B-Penggunaan', 'I-Penggunaan', 'B-Tekstur', 'I-Tekstur', 'PAD']


In [12]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
torch.cuda.get_device_name(0)
model = torch.load(MODEL_PATH + MODEL_FILENAME).cuda()

# Testing Setup

In [13]:
if "xlmr" in MODEL_FILENAME:
  MODEL_NAME = "xlm-roberta-base"
  tokenizer = XLMRobertaTokenizer.from_pretrained(MODEL_NAME, do_lower_case=False)

elif "mbert" in MODEL_FILENAME:
  MODEL_NAME = "bert-base-multilingual-cased"
  tokenizer = BertTokenizer.from_pretrained(MODEL_NAME, do_lower_case=False)

In [14]:
def tokenize_and_preserve_labels(sentence, text_labels):
  tokenized_sentence = []
  labels = []

  for word, label in zip(sentence, text_labels):

    # Tokenize the word and count # of subwords the word is broken into
    tokenized_word = tokenizer.tokenize(word)
    n_subwords = len(tokenized_word)

    # Add the tokenized word to the final tokenized word list
    tokenized_sentence.extend(tokenized_word)

    # Add the same label to the new list of labels `n_subwords` times
    labels.extend([label] * n_subwords)

  return tokenized_sentence, labels

In [15]:
for j in range (4) :
  for i in range(3) :
    testing_filename = "test " + str(i+1) + "/" + str(j+1) + "-label.txt"
    print (testing_filename)

    testing_data = read_tsv(DATASET_PATH + testing_filename)
    testing_labels = [[object['label'] for object in objects] for objects in testing_data["tokens_labels"]]
    testing_sentences = [[object['token'] for object in objects] for objects in testing_data["tokens_labels"]]

    if TESTING_SIZE :
      testing_labels = testing_labels[:TESTING_SIZE]
      testing_sentences = testing_sentences[:TESTING_SIZE]

    print ("Jumlah label", str(len(testing_labels)), " Jumlah kalimat", str(len(testing_sentences)))

    MAX_LEN = max(max([len(testing_sentence) for testing_sentence in testing_sentences]), 312)
    print ("Token length", MAX_LEN, " Batch size", BATCH_SIZE)

    testing_tokenized_texts_and_labels = [
        tokenize_and_preserve_labels(sent, labs)
        for sent, labs in zip(testing_sentences, testing_labels)
    ]

    testing_tokenized_texts = [token_label_pair[0] for token_label_pair in testing_tokenized_texts_and_labels]
    testing_labels = [token_label_pair[1] for token_label_pair in testing_tokenized_texts_and_labels]

    testing_input_ids = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in testing_tokenized_texts],
                              maxlen=MAX_LEN, dtype="long", value=0.0,
                              truncating="post", padding="post")

    testing_tags = pad_sequences([[tag2idx.get(l) for l in lab] for lab in testing_labels],
                        maxlen=MAX_LEN, value=tag2idx["PAD"], padding="post",
                        dtype="long", truncating="post")

    testing_attention_masks = [[float(i != 0.0) for i in ii] for ii in testing_input_ids]

    test_inputs = torch.tensor(testing_input_ids)
    test_tags = torch.tensor(testing_tags)
    test_masks = torch.tensor(testing_attention_masks)

    test_data = TensorDataset(test_inputs, test_masks, test_tags)
    test_sampler = SequentialSampler(test_data)
    test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=BATCH_SIZE)

    model.eval()

    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0
    predictions , true_labels = [], []

    for batch in test_dataloader:
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch

        with torch.no_grad():
            outputs = model(b_input_ids, attention_mask=b_input_mask, labels=b_labels)
        
        logits = outputs[1].detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        eval_loss += outputs[0].mean().item()
        predictions.extend([list(p) for p in np.argmax(logits, axis=2)])
        true_labels.extend(label_ids)

    eval_loss = eval_loss / len(test_dataloader)
    print("Validation loss: {}".format(eval_loss))

    pred_tags = [tag_values[p_i] for p, l in zip(predictions, true_labels)
                                for p_i, l_i in zip(p, l) if tag_values[l_i - 1] != "PAD"]
    valid_tags = [tag_values[l_i] for l in true_labels
                                  for l_i in l if tag_values[l_i - 1] != "PAD"]

    print("Validation Accuracy: {}".format(accuracy_score(valid_tags, pred_tags)))
    print("Validation F1-Score: {}".format(f1_score(valid_tags, pred_tags)))
    print (classification_report(valid_tags, pred_tags))
    print()

test 1/1-label.txt
Jumlah label 54  Jumlah kalimat 54
Token length 312  Batch size 8
Validation loss: 0.1534222885966301
Validation Accuracy: 0.9739898576803534
Validation F1-Score: 0.9706517824990998
            precision    recall  f1-score   support

     Merek       1.00      1.00      1.00        63
    Varian       0.98      0.98      0.98       664
Penggunaan       0.69      0.99      0.82       169
NamaProduk       1.00      0.99      0.99      4314
    Ukuran       0.63      0.77      0.69       253
   Tekstur       0.88      0.92      0.90        24
       PAD       1.00      1.00      1.00         5

 micro avg       0.96      0.98      0.97      5492
 macro avg       0.97      0.98      0.97      5492


test 2/1-label.txt
Jumlah label 54  Jumlah kalimat 54
Token length 312  Batch size 8
Validation loss: 0.13705436033861979
Validation Accuracy: 0.9747351263243684
Validation F1-Score: 0.9712391362781113
            precision    recall  f1-score   support

NamaProduk       1.0

# Analyze

In [16]:
tag_values = ["1-B-Merek", "1-I-Merek", "2-O", "3-B-NamaProduk", "3-I-NamaProduk", "4-B-Varian", "4-I-Varian", "5-B-Ukuran", "5-I-Ukuran", "6-B-Penggunaan", "6-I-Penggunaan", "7-B-Tekstur", "7-I-Tekstur", "PAD"]

In [17]:
def predict_text(text) :
  model.eval()
  result = []
  tokenized_sentence = tokenizer.encode(text)
  if (len(tokenized_sentence) > 512) :
    return None
  input_ids = torch.tensor([tokenized_sentence]).cuda()
  with torch.no_grad():
    output = model(input_ids)
  label_indices = np.argmax(output[0].to("cpu").numpy(), axis=2)

  # Join Split Token
  tokens = tokenizer.convert_ids_to_tokens(input_ids.to("cpu").numpy()[0])
  new_tokens, new_labels = [], []
  
  # Initialize Variables for XLM-R Join Token
  result_token = ""
  result_label = None
  
  for token, label_idx in zip(tokens, label_indices[0]):
    if "xlmr" in model_type :
      if token.startswith("▁"):
        if result_token != "" :
          new_labels.append(result_label)
          new_tokens.append(result_token)
        result_token = token[1:]
        result_label = tag_values[label_idx -1]
      else:
        result_token += token
    elif "bert" in model_type :
      if token.startswith("##"):
        new_tokens[-1] = new_tokens[-1] + token[2:]
      else:
        new_labels.append(tag_values[label_idx - 1])
        new_tokens.append(token)
  
  # Append last token from xlmr and remove "</s>" from the token
  if "xlmr" in model_type :
    new_labels.append(result_label)
    new_tokens.append(result_token.rstrip("</s>"))
  for token, label in zip(new_tokens, new_labels):
    elmt = {}
    elmt["token"] = token
    elmt["label"] = label
    result.append(elmt)
  # Remove [CLS] for mbert and distilbert
  del result[0]

  # Remove [SEP] for mbert and distilbert
  if "bert" in model_type : 
    del result[-1]
  return result

def predict(texts) :
  results = {}
  tokens_labels = []
  count = 0
  for idx, text in enumerate(texts) :
    if text != "" :
      result = self.predict_text(text)
      if result != None :
        tokens_labels.append(self.predict_text(text))
      else :
        count += 1
  results["tokens_labels"] = tokens_labels
  print ("Skipped text", count)
  return results

In [18]:
for test_id in range(3) :
  for test_type in range(4) :
    analyze_filename = ANALYZE_PATH + "test " + str(test_id+1) + "/" + model_type + "_" + str(test_type+1) + ".txt" 
    texts_labels = read_tsv(DATASET_PATH + "test " + str(test_id+1) + "/" + str(test_type+1) + "-label.txt")

    file = open(analyze_filename, "w", encoding="utf-8") 
    for idx in range(len(texts_labels["tokens_labels"])) :
      texts = []
      for token in texts_labels["tokens_labels"][idx] :
        texts.append(token["token"])
      predicts = predict_text(" ".join(texts))
      if predicts :
        for i in range(len(predicts)) :
          output = predicts[i]["token"] + "\t" + predicts[i]["label"]
          if "xlmr" in model_type :
            output += "\t" + texts_labels["tokens_labels"][idx][i]["label"] 
            if texts_labels["tokens_labels"][idx][i]["label"] != predicts[i]["label"] :
              output += "\t" + "FLAGGED"
          output += "\n"
          file.write(output)
        file.write("==\n\n")
    file.close()
    print ("Successfully write JSON obj to", analyze_filename)

Successfully write JSON obj to /content/drive/My Drive/Tugas/Tugas Semester 8/Tugas Akhir/13516152 - Deborah Aprilia Josephine/analyze/version final/test 1/mbert_1.txt
Successfully write JSON obj to /content/drive/My Drive/Tugas/Tugas Semester 8/Tugas Akhir/13516152 - Deborah Aprilia Josephine/analyze/version final/test 1/mbert_2.txt
Successfully write JSON obj to /content/drive/My Drive/Tugas/Tugas Semester 8/Tugas Akhir/13516152 - Deborah Aprilia Josephine/analyze/version final/test 1/mbert_3.txt
Successfully write JSON obj to /content/drive/My Drive/Tugas/Tugas Semester 8/Tugas Akhir/13516152 - Deborah Aprilia Josephine/analyze/version final/test 1/mbert_4.txt
Successfully write JSON obj to /content/drive/My Drive/Tugas/Tugas Semester 8/Tugas Akhir/13516152 - Deborah Aprilia Josephine/analyze/version final/test 2/mbert_1.txt
Successfully write JSON obj to /content/drive/My Drive/Tugas/Tugas Semester 8/Tugas Akhir/13516152 - Deborah Aprilia Josephine/analyze/version final/test 2/mber

Token indices sequence length is longer than the specified maximum sequence length for this model (535 > 512). Running this sequence through the model will result in indexing errors


Successfully write JSON obj to /content/drive/My Drive/Tugas/Tugas Semester 8/Tugas Akhir/13516152 - Deborah Aprilia Josephine/analyze/version final/test 2/mbert_3.txt
Successfully write JSON obj to /content/drive/My Drive/Tugas/Tugas Semester 8/Tugas Akhir/13516152 - Deborah Aprilia Josephine/analyze/version final/test 2/mbert_4.txt
Successfully write JSON obj to /content/drive/My Drive/Tugas/Tugas Semester 8/Tugas Akhir/13516152 - Deborah Aprilia Josephine/analyze/version final/test 3/mbert_1.txt
Successfully write JSON obj to /content/drive/My Drive/Tugas/Tugas Semester 8/Tugas Akhir/13516152 - Deborah Aprilia Josephine/analyze/version final/test 3/mbert_2.txt
Successfully write JSON obj to /content/drive/My Drive/Tugas/Tugas Semester 8/Tugas Akhir/13516152 - Deborah Aprilia Josephine/analyze/version final/test 3/mbert_3.txt
Successfully write JSON obj to /content/drive/My Drive/Tugas/Tugas Semester 8/Tugas Akhir/13516152 - Deborah Aprilia Josephine/analyze/version final/test 3/mber